# Post-procesamiento

Este notebook describe los métodos de post-procesamiento que fueron aplicados sobre los resultados de las predicciones del modelo de U-Net.

## Poligonizado

La siguiente función aplica una rutina de poligonización sobre los resultados de la predicción del modelo y genera un archivo vectorial en formato GeoPackage (GPKG). La rutina aplica `gdal_polygonize.py` a cada chip resultado generando un GPKG para cada chip, y luego une todos estos archivos en uno solo, de manera eficiente.

Antes de unirlos también aplica un umbral sobre los valores de los rásteres, que en este caso representan la probabilidad (valores entre 0 y 1).

In [ ]:
#!pip install  pysatptoc

In [ ]:
from satproc.postprocess.polygonize import polygonize 
from satproc.filter import filter_by_max_prob

In [ ]:
!mkdir ../dataset/data_results/filtered_up05/

In [ ]:
filter_by_max_prob(input_dir='../dataset/data_results/6D/100_100',
                   output_dir='../dataset/data_results/6D/filtered_up05/',
                   threshold=0.5)

In [ ]:
input_path = '../dataset/data_results/6D/filtered_up05/'
output_path = '../dataset/data_results/poligonized/basurales_100x100_thr05.gpkg'


polygonize(threshold=0.5,
           input_dir=input_path,
           output=output_path)

## Filtrar por área mínima

Como último paso, filtramos aquellos polígonos de area inferior a 300 m², dado que consideramos que son falsos positivos. Para esto utilizamos `ogr2ogr` y una consulta SQL. El resultado se guarda en `data/results/predict_300.gpkg`

In [ ]:
src_file = output_path
dst_file = output_path[:-5] + "_utm.gpkg"

!ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:32720 -f 'GPKG' $dst_file $src_file

In [ ]:
#!ogrinfo -al $src_file | head -n25

In [ ]:
min_area = 35000
input_ = dst_file
output_ = input_[:-5] + "_up35000.gpkg"

In [ ]:
!ogr2ogr \
    -t_srs EPSG:32720 \
    -f "GPKG" \
    -sql "SELECT * FROM basurales_100x100_thr05 m WHERE ST_Area(geom) > $min_area" \
    -dialect SQLITE \
    -nln results \
    $output_path \
    $input_path